<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       VantageCloud Lake Path Analyzer
  <br>
       <img id="teradata-logo" src="../../images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>

<br>

<b style = 'font-size:24px;font-family:Arial;color:#00233C'>Demo 0 - Foreign Table Setup</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook sets up the Foreign Table "demo_ofs.Digital_Retail_Events" that is used for Path Analyzer UI demo.</p>
<hr>

In [1]:
# /* -------------------------------------------------------- */
# -- Perform this as SYSDBA.
# -- Load demo_ofs.Digital_Retail_Events from source file
# /* -------------------------------------------------------- */

import teradatasql, json

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    
    try:
        cur.execute('DROP TABLE demo_ofs.Digital_Retail_Events')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    
    qry = '''
    CREATE MULTISET TABLE demo_ofs.Digital_Retail_Events

     (
      Entity_Id INTEGER,
      datestamp TIMESTAMP(6),
      event VARCHAR(19) CHARACTER SET LATIN NOT CASESPECIFIC,
      session_id INTEGER)
    NO PRIMARY INDEX;
    '''
    cur.execute(qry)
    
    qry = f'''
    Insert into demo_ofs.Digital_Retail_Events 
    SELECT Entity_Id, datestamp, event, session_id
    FROM
            (
                LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/DIGITAL_RETAIL_EVENTS/manifest.json'
                MANIFEST = 'TRUE'
                AUTHORIZATION = Repositories.PubAuth
            ) D;'''
    cur.execute(qry)

Copyright 2023, Teradata Corporation